In [56]:
import requests
import pandas as pd
from datetime import datetime

In [57]:
SERVERS = [
    {"name": "Server1", "url": "http://127.0.0.1:8001", "type": "temperature"},
    {"name": "Server2", "url": "http://127.0.0.1:8002", "type": "temperature"},
    {"name": "PDU1", "url": "http://127.0.0.1:9001", "type": "power"},
    {"name": "Server1", "url": "http://127.0.0.1:8001", "type": "fan_speed"},
    {"name": "Server2", "url": "http://127.0.0.1:8002", "type": "fan_speed"}
]

ENDPOINTS = {
    "temperature": "/redfish/v1/Chassis/1U/Thermal",
    "power": "/redfish/v1/Chassis/0/Power",
    "fan_speed": "/redfish/v1/Chassis/1U/Thermal"
}

# 取得開始時刻（ファイル名用）
START_TIME = datetime.now().strftime("%Y%m%d_%H%M%S")

# 出力CSVファイル名
CSV_FILE = f"server_pdu_data_{START_TIME}.csv"

In [58]:
def fetch_data_with_timeout(server):
    """APIリクエストを実行し、タイムアウトやエラーを適切に処理"""
    url = f"{server['url']}{ENDPOINTS[server['type']]}"
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    try:
        response = requests.get(url, verify=False, timeout=5)  # SSL無視、タイムアウト5秒
        response.raise_for_status()
        return response.json(), url  # 成功時はデータとURLを返す

    except requests.exceptions.RequestException as e:
        print(f"❌ Error accessing {url}: {e}")
        return [{
            "Timestamp": timestamp,
            "Device": server["name"],
            "IP": server["url"],  # ここを追加
            "URL": url,  # URLも保持
            "Type": "Error",
            "Name": "N/A",  # エラー時のデータを統一
            "Value": "N/A",
            "Unit": "N/A",
            "Error Message": str(e)  # エラーメッセージを記録
        }], url


In [59]:
def fetch_data_with_timeout(server):
    """APIリクエストを実行し、タイムアウトやエラーを適切に処理"""
    url = f"{server['url']}{ENDPOINTS[server['type']]}"
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    try:
        response = requests.get(url, verify=False, timeout=5)  # SSL無視、タイムアウト5秒
        response.raise_for_status()
        return response.json(), url  # 取得成功時はデータとURLを返す

    except requests.exceptions.RequestException as e:
        print(f"❌ Error accessing {url}: {e}")
        return [{
            "Timestamp": timestamp,
            "Device": server["name"],
            "IP": server["url"],
            "URL": url,  # URLを追加
            "Type": "Error",
            "Name": None,
            "Value": None,
            "Unit": None,
            "Error Message": str(e)  # エラーメッセージを記録
        }], url

In [60]:
def extract_cpu_temperature(data, device_name, url):
    """Redfish APIのレスポンスからCPUの温度データを取得"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    results = []

    for sensor in data.get("Temperatures", []):
        if "CPU" in sensor.get("Name", ""):
            results.append({
                "Timestamp": timestamp,
                "Device": device_name,
                "URL": url,  # 追加
                "Type": "Temperature",
                "Name": sensor["Name"],
                "Value": sensor["ReadingCelsius"],
                "Unit": "Celsius"
            })

    return results

In [61]:
def extract_pdu_power(data, device_name, url):
    """Redfish APIのレスポンスからPDUの消費電力を取得"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    results = []

    for power_ctrl in data.get("PowerControl", []):
        results.append({
            "Timestamp": timestamp,
            "Device": device_name,
            "URL": url,  # 追加
            "Type": "Power",
            "Name": power_ctrl["Name"],
            "Value": power_ctrl["PowerConsumedWatts"],
            "Unit": "Watts"
        })

    return results

In [62]:
def extract_pdu_power(data, device_name, url):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    results = []

    for power_ctrl in data.get("PowerControl", []):
        results.append({
            "Timestamp": timestamp,
            "Device": device_name,
            "URL": url,
            "Type": "Power",
            "Name": power_ctrl["Name"],
            "Value": power_ctrl["PowerConsumedWatts"],
            "Unit": "Watts"
        })

    return results

In [65]:
def extract_fan_speed(data, device_name, url):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    results = []

    for fan in data.get("Fans", []):
        results.append({
            "Timestamp": timestamp,
            "Device": device_name,
            "URL": url,
            "Type": "FanSpeed",
            "Name": fan["Name"],
            "Value": fan["Reading"],
            "Unit": fan["ReadingUnits"]
        })

    return results

In [66]:
def fetch_and_display_data():
    """全サーバーと PDU からデータを取得し、DataFrame で表示"""
    all_data = []

    for server in SERVERS:
        print(f"\nFetching data from {server['name']} ({server['url']}) ...")
        
        data, url = fetch_data_with_timeout(server)
        
        # もし data がエラー情報を含んでいる場合、そのまま追加
        if isinstance(data, list) and data and data[0].get("Type") == "Error":
            all_data.extend(data)
            continue  # エラーならスキップ

        # データ取得が成功した場合、適切な処理関数を呼ぶ
        if server["type"] == "temperature":
            all_data.extend(extract_cpu_temperature(data, server["name"], url))
        elif server["type"] == "power":
            all_data.extend(extract_pdu_power(data, server["name"], url))
        elif server["type"] == "fan_speed":  # FanSpeed の処理追加
            all_data.extend(extract_fan_speed(data, server["name"], url))

    if not all_data:
        print("⚠ No data found.")
        return pd.DataFrame()  # 空の DataFrame を返す

    df = pd.DataFrame(all_data)
    
    # CSVに保存
    df.to_csv(CSV_FILE, index=False)
    
    print(f"\n✅ Data saved to {CSV_FILE}")
    
    return df


In [67]:
df = fetch_and_display_data()
df


Fetching data from Server1 (http://127.0.0.1:8001) ...

Fetching data from Server2 (http://127.0.0.1:8002) ...

Fetching data from PDU1 (http://127.0.0.1:9001) ...
❌ Error accessing http://127.0.0.1:9001/redfish/v1/Chassis/0/Power: HTTPConnectionPool(host='127.0.0.1', port=9001): Read timed out. (read timeout=5)

Fetching data from Server1 (http://127.0.0.1:8001) ...

Fetching data from Server2 (http://127.0.0.1:8002) ...

✅ Data saved to server_pdu_data_20250211_100737.csv


,Timestamp,Device,URL,Type,Name,Value,Unit,IP,Error Message
0,2025-02-11 10:08:11,Server1,http://127.0.0.1:8001/redfish/v1/Chassis/1U/Th...,Temperature,CPU1 Temp,111.0,Celsius,NaN,NaN
1,2025-02-11 10:08:11,Server1,http://127.0.0.1:8001/redfish/v1/Chassis/1U/Th...,Temperature,CPU2 Temp,112.0,Celsius,NaN,NaN
2,2025-02-11 10:08:12,Server2,http://127.0.0.1:8002/redfish/v1/Chassis/1U/Th...,Temperature,CPU1 Temp,221.0,Celsius,NaN,NaN
3,2025-02-11 10:08:12,Server2,http://127.0.0.1:8002/redfish/v1/Chassis/1U/Th...,Temperature,CPU2 Temp,222.0,Celsius,NaN,NaN
4,2025-02-11 10:08:12,PDU1,http://127.0.0.1:9001/redfish/v1/Chassis/0/Power,Error,None,NaN,None,http://127.0.0.1:9001,"HTTPConnectionPool(host='127.0.0.1', port=9001..."
5,2025-02-11 10:08:17,Server1,http://127.0.0.1:8001/redfish/v1/Chassis/1U/Th...,FanSpeed,BaseBoard System Fan,2100.0,RPM,NaN,NaN
6,2025-02-11 10:08:17,Server1,http://127.0.0.1:8001/redfish/v1/Chassis/1U/Th...,FanSpeed,BaseBoard System Fan Backup,2050.0,RPM,NaN,NaN
7,2025-02-11 10:08:17,Server2,http://127.0.0.1:8002/redfish/v1/Chassis/1U/Th...,FanSpeed,BaseBoard System Fan,2100.0,RPM,NaN,NaN
8,2025-02-11 10:08:17,Server2,http://127.0.0.1:8002/redfish/v1/Chassis/1U/Th...,FanSpeed,BaseBoard System Fan Backup,2050.0,RPM,NaN,NaN


In [ ]:
# def extract_fan_speed(data, device_name):
#     timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#     results = []

#     fan = data.get("Fans")  # ここでデータを取得

#     if not fan:  # データがなければ空リストを返す
#         return []

#     results.append({
#         "Timestamp": timestamp,
#         "Device": device_name,
#         "Type": "FanSpeed",
#         "Name": fan["Name"],
#         "Value": fan["Reading"],
#         "Unit": fan["ReadingUnits"]
#     })
    
#     return results
